#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2018


# Homework 2:  Link Analysis -- HITS + SEO

### 100 points [5% of your final grade]

### Due: Sunday, February 25, 2018 by 11:59pm

*Goals of this homework:* Explore real-world challenges of building a graph (in this case, from tweets), implement and test HITS algortihm over this graph, and investigate factors that impact a page's rank on Google and Bing.

*Submission Instructions:* To submit your homework, rename this notebook as YOUR_UIN_hw2.ipynb. Submit this notebook via ecampus. Your notebook should be completely self-contained, with the results visible in the notebook. 

*Late submission policy:* For this homework, you may use up to three of your late days, meaning that no submissions will be accepted after Wednesday, February 28, 2018 at 11:59pm.

# Part 1: HITS (70 points)

## A re-Tweet Graph

In this assignment, we're going to adapt the classic HITS approach to allow us to find not the most authoritative web pages, but rather to find significant Twitter users. So, instead of viewing the world as web pages with hyperlinks (where pages = nodes, hyperlinks = edges), we're going to construct a graph of Twitter users and their retweets of other Twitter users (so user = node, retweet of another user = edge). Over this Twitter-user graph, we can apply the HITS approach to order the users by their hub-ness and their authority-ness.

Here is a toy example. Suppose you are given the following four retweets:

* **userID**: diane, **text**: "RT ", **sourceID**: bob
* **userID**: charlie, **text**: "RT Welcome", **sourceID**: alice
* **userID**: bob, **text**: "RT Hi ", **sourceID**: diane
* **userID**: alice, **text**: "RT Howdy!", **sourceID**: parisa

There are four short tweets retweeted by four users. The retweet between users form a directed graph with five nodes and four edges. E.g., the "diane" node has a directed edge to the "bob" node.

You should build a graph by parsing the tweets in the file we provide called *HITS.json*.

**Notes:**

* You may see some weird characters in the content of tweets, just ignore them. 
* The edges are weighted and directed. If Bob retweets Alice's tweets 10 times, there is an edge from Bob to Alice with weight 10, but there is not an edge from Alice to Bob.
* If a user retweets herself, ignore it.
* Correctly parsing screen_name in a tweet is error-prone. Use the id of the user (this is the user who is re-tweeting) and the id of the user in the retweeted_status field (this is the user who is being re-tweeted; that is, this user created the original tweet).
* Later you will need to implement the HITS algorithm on the graph you build here.


In [2]:
# your code here 
import numpy as np
import os
import json
import urllib2
import networkx as nx
with open('/Users/huangyian/Desktop/HITS.json', 'r') as f:
    file_lines = [''.join([x.strip(), ',', '\n']) for x in f.readlines()]
# print len(file_lines)
output = open('/Users/huangyian/Desktop/h.json', 'w')
output.write('[')
with output as f:
    for i in range(0, len(file_lines) - 1):
        f.writelines(file_lines[i]) 
    lastline = file_lines[len(file_lines) - 1].strip()[:-1]
    f.writelines(lastline)
    f.write(']')
output.close() 

with open('/Users/huangyian/Desktop/outfile.json', 'r') as f:
    allfile = json.load(f)

G = nx.DiGraph()
for eachUser in allfile:
    if (eachUser['user']['id']) == (eachUser['retweeted_status']['user']['id']):
        continue
    else:
        source = eachUser['user']['id']
        user = eachUser['retweeted_status']['user']['id']
#         G.add_edge((eachUser['user']['id']),(eachUser['retweeted_status']['user']['id']),weight = eachUser['retweeted_status']['retweet_count']
        if G.has_edge(source, user):
            w = G[source][user]['weight']
            G.add_edge(source,user,weight = w + 1)
        else:
            G.add_edge(source,user,weight = 1)
    
adjacency_matrix = nx.to_numpy_matrix(G)
# //3052705430 and 3039321886.
row = len(G)
l = G.nodes()
nodes = np.asarray(l)
dictionary = {}
for i in range(0, row):
    dictionary[i] = nodes[i]
print adjacency_matrix



[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


We will not check the correctness of your graph. However, this will affect the HITS results later.

## HITS Implementation

Your program will return the top 10 users with highest hub and authority scores. The **output** should be like:

Hub Scores

* user1 - score1
* user2 - score2
* ...
* user10 - score10

Authority Scores

* user1 - score1
* user2 - score2
* ...
* user10 - score10

You should follow these **rules**:

* Assume all nodes start out with equal scores.
* It is up to you to decide when to terminate the HITS calculation.
* There are HITS implementations out there on the web. Remember, your code should be **your own**.


**Hints**:
* If you're using the matrix style approach, you should use [numpy.matrix](https://docs.scipy.org/doc/numpy/reference/generated/numpy.matrix.html).
* Scipy is built on top of Numpy and has support for sparse matrices. You most likely will not need to use Scipy unless you'd like to try out their sparse matrices.
* If you choose to use Numpy (and Scipy), please make sure your Anaconda environment include their latest versions.
* Test your parsing and HITS calculations using a handful of tweets, before moving on to the entire file we provide.
* We will evaluate the user ranks you provide as well as the quality of your code. So make sure that your code is clear and readable.

In [3]:
# your code here
import heapq
# Transpose of adjacency matrix
transposed_adjacency_matrix = adjacency_matrix.transpose()

# Initialize the hubs and authorities
hubs = np.ones((row, 1))
authorities = np.ones((row, 1))

iterate = 100
while (iterate != 0):
    hubs = adjacency_matrix * authorities 
    authorities = transposed_adjacency_matrix * hubs
#     Normalize hubs and authorities matrices
    hubs = hubs / np.linalg.norm(hubs)
    authorities = authorities / np.linalg.norm(authorities)
    iterate = iterate - 1

print "Top 20 authority scores:"
no = 1
authorityList = authorities.getA1()
for num in heapq.nlargest(20, authorityList):
    auth_ind = np.where(authorityList == num)[0][0]
    print no, ".", dictionary.get(auth_ind), authorityList[auth_ind]
    no += 1
    
print "Top 20 hubs scores:"
no1 = 1
hubList = hubs.getA1()
for num in heapq.nlargest(20, hubList):
    hub_ind = np.where(hubList == num)[0][0]
    print no1, ".", dictionary.get(hub_ind), hubList[hub_ind]
    no1 += 1
# # print hubs


Top 20 authority scores:
1 . 3042570996 0.5446248048789051
2 . 3065514742 0.49321659986878863
3 . 1638625987 0.4439307063060393
4 . 3077733683 0.28659130814272554
5 . 3039321886 0.22433355505187036
6 . 3077695572 0.12188415726146554
7 . 3019659587 0.11322781827450253
8 . 1358345766 0.09803152133129721
9 . 3061155846 0.09398530780220564
10 . 3092580049 0.09367926178547767
11 . 571198546 0.0815583527765113
12 . 3068694151 0.0750801646278131
13 . 3058933933 0.07165323065160539
14 . 3154266823 0.07004226946434818
15 . 3126638673 0.06305455799411037
16 . 3082049188 0.06207068854237583
17 . 610166901 0.05812165502285397
18 . 3083157411 0.05776406549579737
19 . 3190712044 0.05138631366298336
20 . 3086054424 0.04639484142299678
Top 20 hubs scores:
1 . 3068706044 0.6231186081225087
2 . 3093940760 0.29616066155009024
3 . 2194518394 0.259872564249747
4 . 2862783698 0.2025856567488808
5 . 3092183276 0.170518449128319
6 . 3029724797 0.16698984616196605
7 . 2990704188 0.14773329728408705
8 . 3001500

# Part 2: Search Engine Optimization (30 + 5 points)

For this part, your goal is to put on your "[search engine optimization](https://en.wikipedia.org/wiki/Search_engine_optimization)" hat. Your job is to create a webpage that scores highest for the query: **kbeznak parmatonic** --- two terms, lower case, no quote. As of today (Feb 16, 2018), there are no hits for this query on either Google or Bing. Based on our discussions of search engine ranking algorithms, you know that several factors may impact a page's rank. Your goal is to use this knowledge to promote your own page to the top of the list.

What we're doing here is a form of [SEO contest](https://en.wikipedia.org/wiki/SEO_contest). While you have great latitude in how you approach this problem, you are not allowed to engage in any unethical or illegal behavior. Please read the discussion of "white hat" versus "black hat" SEO over at [Wikipedia](https://en.wikipedia.org/wiki/Search_engine_optimization#White_hat_versus_black_hat_techniques).


**Rules of the game:**

* Somewhere in the page (possibly in the non-viewable source html) you must include your name or some other way for us to identify you (e.g., your NetID, but not the UIN!).
* Your target page may only be a TAMU student page, a page on your own webserver, a page on a standard blog platform (e.g., wordpress), or some other primarily user-controlled page
* Your target page CAN NOT be a twitter account, a facebook page, a Yahoo Answers or similar page
* No wikipedia vandalism
* No yahoo/wiki answers questions
* No comment spamming of blogs
* If you have concerns/questions/clarifications, please post on Piazza and we will discuss

For your homework turnin for this part, you should provide us the URL of your target page and a brief discussion (2-4 paragraphs) of the strategies you are using. We will issue the query and check the rankings at some undetermined time in the next couple of weeks. You might guess that major search engines take some time to discover and integrate new pages: if I were you, I'd get a target page up immediately.

**Grading:**

* 5 points for providing a valid URL
* 20 points for a well-reasoned discussion of your strategy
* 5 points for your page appearing in the search results by Google or Bing (no matter how is the ranking)

** Bonus: **
* 1 point for your page appearing in the top-20 on Google or Bing
* 1 more point for your page appearing in the top-10 on Google or Bing
* 1 more point for your page appearing in the top-5 on Google or Bing
* 2 more points for your page being ranked first by Google or Bing. And, a vigorous announcement in class, and a high-five for having the top result!

What's the URL of your page?

https://hdx1993.github.io/kbeznak-parmatonic/

What's your strategy? (2-4 paragraphs)


I use Github io the build the kbeznak parmatonic website: 
    https://hdx1993.github.io/kbeznak-parmatonic/
To improve my page rank, firstly, I tried to improve my content’s quality. Instead of copying sentences from the internet, I wrote paragraphs on my own, as well as quoting some Wikipedia items such as SEO definition into my content. I assume the phrase kbeznak parmatonic is related to CSE670, information retrieval, Texas A&M University, Professor Caverlee, etc. Therefore, I tried to add links of these topics in my website and avoid irrelevant topics in my website. Additionally, to be a website with good quality, I update my website regularly. 

Also, I focus on inlinks and backlinks. My website has hyperlink toward good websites such as google, YouTube and Wikipedia, and also contains links towards other useful links, such as YouTube videos on kbeznak parmatonic. For the backlinks, I create blogs from other good websites because we are not allowed to use social media like facebook. For example, I wrote another blog on WordPress and Douban(a famous story-telling website in China) talking about kbeznak parmatonic. I pointed to https://hdx1993.github.io/kbeznak-parmatonic/ in my new blogs. Through these method, my page rank score will be improved.

If I want website to be shown on google, adding my websites (both GitHub one: https://hdx1993.github.io/kbeznak-parmatonic/ and WordPress one: https://whatiskbeznakparmatonic.wordpress.com/) on google search console is necessary. After uploading html verification files or adding a meta tag to my site's home page, I could see my website on google if I google: site: https://hdx1993.github.io/kbeznak-parmatonic/.